# Week 2 tutorial: Logistic regression

### Introduction
In this exercise, you will implement logistic regression and get to see it work on data. After implementing a simple case, we will see how to repeat our work using scikit-learn, then work on more complex cases that require feature mapping and regularization. Hopefully you have watched the video lectures associated with this week. **Please keep the function declarations as they are.**

To get started with this exercise, you will need to download the Week 2 folder from the github or dropbox.
<br>

### External files included for this exercise
 - $\texttt{ex2data1.txt}$ - Dataset for linear regression with one variable
 - $\texttt{ex2data2.txt}$ - Dataset for linear regression with multiple variables
 - Week_2_tute_P1 - Part one of this week's exercise.
 - Week_2_tute_P2 - Part two of this week's exercise.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 1 - Logistic Regression

In this part of the exercise, you will build a logistic regression model to predict whether a student gets admitted into a university.

Suppose that you are the administrator of a university department and you want to determine each applicant’s chance of admission based on their results on two exams. You have historical data from previous applicants that you can use as a training set for logistic regression. For each training example, you have the applicant’s scores on two exams and the admissions decision.

Your task is to build a classification model that estimates an applicant’s probability of admission based the scores from those two exams. This outline and the framework code in this notebook will guide you through the exercise.

### 1.1 - Visualizing the data
Before starting to implement any learning algorithm, it is always good to visualize the data if possible. This next cell will load the data and display it on a 2-d plot. The plotting function will display a figure where the axes are the two exam scores, and the positive and negative admission examples are shown with different markers. <br>
**Optional exercise:** Take a few moments to change the arguments in the function calls to `plot(...)` to ensure you understand what they each do. 

In [ ]:
data = np.loadtxt('./data/ex2data1.txt',delimiter=',')
X = data[:,0:2] 
m = X.shape[0]
one = np.ones((m,1))
X = np.concatenate([one,X],axis=1)
y = data[:,-1].reshape((m,1))
# A fun lil list comprehension to get the relevant row indexs for each case
pos = [idx for (idx,val) in enumerate(y) if val==1]
neg = [idx for (idx,val) in enumerate(y) if val==0]

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.plot(X[pos,1],X[pos,2],'kx',MarkerSize=8,label='Admitted')
plt.plot(X[neg,1],X[neg,2],'ko',MarkerFaceColor='yellow',MarkerSize=8,label='Not admitted')
plt.xlim([30,100])
plt.ylim([30,100])
plt.xlabel("Exam 1 score")
plt.ylabel("Exam 2 score")
plt.legend()
plt.show()

### 1.2 - Implementation
#### 1.2.1 - Sigmoid function
Before we start with the actual cost function, let's recall that the logistic regression hypothesis is defined as:
\begin{equation*}
h_\theta = g(x\theta)
\end{equation*}
where $g(z)$ is the sigmoid function, 
\begin{equation*}
g(z) = \frac{1}{1+\mathrm{e}^{-z}}
\end{equation*}
$\theta$ is a (n,1) matrix, and $x$ is a single example (1,n).
<br>
Your first step is to implement the sigmoid function below. When you are finished, try testing a few values by calling `sigmoid(x)`. For large values of $x$, the sigmoid should be close to 1, while for large negative values, the sigmoid should be clost to 0. Evaluating `sigmoid(0)` should give exactly 0.5. 
<br>
**Exercise: Finish the function below, such that it will calculate the sigmoid value elementwise for vectors and matricies **

In [ ]:
def sigmoid(x):
    '''
    Computes the sigmoid function elementwise
    
    Input
    ==============
    x : scalar, vector or martrix of values
    
    Returns
    ==============
    g : elementwise sigmoid function. Same shape as x.
    '''
    ### START CODE HERE

    ### END CODE HERE ###
    return g

print(sigmoid(+10))

#### 1.2.2 - Cost function and gradient
Similar to what we did with linear regression, we will now implement the cost function and gradient for logistic regression. Recall that the cost function for logistic regression is the cross entropy
\begin{equation}
J(\theta) = \frac{1}{m}\sum^m_{i=1} \left[
-y^{(i)} \log(h_\theta(x^{(i)})) - (1-y^{(i)}) \log(1-h_\theta(x^{(i)}))
\right]
\end{equation}
and the gradient of the cost is a vector of the same length as $\theta$ where the $j^{\mathrm{th}}$ element (for $j=-,1\dots,n$) is defined as follows:
\begin{equation}
\frac{\partial J(\theta)}{\partial\theta_j} = \frac{1}{m} \sum^m_{i=1} (h_\theta(x^{(i)}) - y^{(i)})x^{(i)}_j
\end{equation}

Here the sums are operating down the $X$ matrix across all trainging examples, and the $j$ refers to particular columns of *features*. Note that while this gradient looks identical to the linear regression gradient, the formula is actually different because linear and logistic regression have different definitions of $h_\theta(x)$. 

**Exercise: Implement the function `compute_cost()` to return both the loss function J, and the gradient.**
<br>
*Hint: A vectorized way of calculating the sum of the elements of a product is an inner product. *

In [ ]:
def compute_cost(theta,X,y):
    '''
    Computs the cost of using theta as the parameters for logistic regression
    and the gradient of the cost with respect ot the parameters. 
    
    Input
    ==============
    theta : (n,1) vector of parameters
    X : (m,n) matrix of training data
    y : (m,1) vector of ground truth data
    
    Returns
    ==============
    J : cost
    grad : (n,1) vector of gradients
    '''
    
    # Initialize useful parameters
    m = X. shape[0]
    # You will need to return the following correctly
    J = 0
    grad = np.zeros(theta.shape)
    
    ### START CODE HERE ###

    ### END CODE HERE ###
    
    return J,grad

In [ ]:
theta=np.zeros((X.shape[1],1))
J,grad = compute_cost(theta,X,y)
print("Initial cost = {:6.4f}".format(J[0,0]))
print("Initial gradiaent =\n",grad)

**Expected output:**<br>
`Initial cost = 0.6931
 Initial gradiaent =
 [[ -0.1       ]
 [-12.00921659]
 [-11.26284221]]`

#### 1.2.3 - Gradient descent

We can implement the gradient descent algorithm exactly as we did in Linear regression. Below however, the cost computation includes the gradient calculation, so the update of theta is much simpler. 

I've left a cell after the gradient descent cell to show how the loss changes over time, and replot the data with your derived linear decision boundary. 

In [ ]:
def gradient_descent(X,y,theta,alpha=0.00001,num_iters=1500):
    '''
    Performs gradient descent to learn theta
    
    Inputs
    ===============
    X: input features
    y: desired ouput values
    alpha: learning rrate
    num_iters: number of iteratons for learning
    
    Returns
    ===============
    theta: final learned weights
    J_history: list of loss over each itteration
    '''
    m = y.shape[0] #Number of training examples
    J_history = []
    
    for i in range(num_iters):
        # Calculate cost and gradient
        J, grad = compute_cost(theta,X,y)
        J_history.append(J[0,0])
        # Update theta
        theta = theta - alpha * grad

    return theta, J_history

initial_theta=np.array([[-13],[0.2],[0.2]])
num_iters=1500
learned_theta, J_hist = gradient_descent(X,y,initial_theta,num_iters=num_iters)

In [ ]:
# Finding decision boundary (this uses an analytical function derived from the 0.5 condition)
plot_x = [np.min(X[:,1]),np.max(X[:,1])]
plot_y = [-1/learned_theta[2,0]*(learned_theta[1,0]*x + learned_theta[0,0]) for x in plot_x]
                
fig1=plt.figure(figsize=(16,6))
ax1 = fig1.add_subplot(1,2,1)
ax1.plot(X[pos,1],X[pos,2],'kx',MarkerSize=8,label='Admitted')
ax1.plot(X[neg,1],X[neg,2],'ko',MarkerFaceColor='yellow',MarkerSize=8,label='Not admitted')
ax1.plot(plot_x,plot_y,'b-',label='Decision boundary')
ax1.legend()
ax1.set_xlim([30,100])
ax1.set_ylim([30,100])
ax1.set_xlabel('Exam 1 score')
ax1.set_ylabel('Exam 2 score')

ax2 = fig1.add_subplot(1,2,2)
ax2.plot([a for a in range(1500)], J_hist,'k.',MarkerSize=5)
ax2.set_xlabel("Number of iterations")
ax2.set_ylabel("Loss")

plt.show()

#### 1.2.4 - Evaluating your function
After learning the parameters, you can use the model to predict whether a particular student will be admitted. For a student with an Exam 1 score of 45 and an Exam 2 score of 85, you should expect to see an admission probability of **?**.
Another way to evaluate the quality of the parameters we have found is to see how well the learned model predicts on our training set. The predict function will produce “1” or “0” predictions given a dataset and a learned parameter vector $\theta$. Then there is a script below to calculate the training accuracy of your model. 
<br>
**Exercise: Comlete the function `predict()` below, to make a predicted y value of 0 or 1.**<br>
*Hint: use the round function or a logical statement (int(True)=1, int(False)=0).*

In [ ]:
def predict(theta,X):
    '''
    Makes a prediction using a learned set of values for theta.
    
    Inputs
    ===============
    theta : (n,1) vector of parameters
    X : (m,n) matrix of training data
    
    Returns
    ===============
    p: (m,1) vector of calclated prediction values
    '''
    m = X. shape[0]
    
    ### START CODE HERE ###

    ### END CODE HERE ###
    return p

p = predict(learned_theta,X)
acc = np.mean(p==y)*100
print("Training accuracy: " ,acc)
print("Expected accuracy (approx): 89.0")

#### 1.2.5 - Blackbox
Excellent work! Now lets take a look at the sklearn implementation. First we'll reimport our data with the intercept column, and use the scikitlearn version of logistic regression on our exam data. 
Calling `linear_model.LogisticRegression()` will return a class, that can be used to fit to our X,y data.

The methods and attributes of this model class are listed here:
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In breif:
- First we initialize the class
- Second we call the .fit() method on the data
- The prediction and scoring is also taken care of in the available methods

In [ ]:
from sklearn.linear_model import LogisticRegression
# Load data: Notice how the shape of y is now (m,) not (m,1)
data = np.loadtxt('./data/ex2data1.txt',delimiter=',')
X = data[:,0:2] 
m = X.shape[0]
one = np.ones((m,1))
X = np.concatenate([one,X],axis=1)
m = X.shape[0]
y = data[:,-1]
# Execute regression
log_reg = LogisticRegression()
log_reg.fit(X,y)
print("Training accuracy: ",log_reg.score(X,y))
print("Even better than before!")

Here is a bit of complicated graphworks. While before we plotted a decision boundary at P=0.5, this shows a contour plot of the probability of admisison given a set of scores. Overlayed are the points colored according to their binary ground truth. The decision boundary is given by the P=0.5 line in green.

In [ ]:
xx, yy = np.mgrid[30:100:0.1, 30:100:0.1]
grid = np.c_[xx.ravel(), yy.ravel()]
one = np.ones((grid.shape[0],1))
grid=np.concatenate((one,grid),axis=1)
probs = log_reg.predict_proba(grid)[:, 1].reshape(xx.shape)

f, ax = plt.subplots(figsize=(8, 6))
contour = ax.contourf(xx, yy, probs, 25, cmap="RdBu",
                      vmin=0, vmax=1)
ax_c = f.colorbar(contour)
ax_c.set_label("$P(y = 1)$")
ax_c.set_ticks([0, .25, .5, .75, 1])
ax.contour(xx, yy, probs, levels=[0.5],vmin=0, vmax=1,colors='green')
ax.scatter(X[:,1], X[:, 2], c=y[:], s=50,
           cmap="RdBu", vmin=-.2, vmax=1.2,
           edgecolor="white", linewidth=1)

ax.set(aspect="equal",
       xlim=(30, 100), ylim=(30, 100),
       xlabel="$X_1$", ylabel="$X_2$")
plt.show()